# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re

In [2]:

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [3]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.has_actual, om.relet_status, om.order_finish_date, om.overdue, tprm.rembursement_status, tprm.purchase_amount
,tol.delivery_province_name, tprm.contract_price, tprm.rent_received, tprm.all_rental, tprm.unearned_rent_revenue
from  db_digua_business.t_postlease_receivables_monitoring   tprm
left join db_digua_business.t_order  om on tprm.order_id=om.id
left join db_digua_business.t_order_logistics tol on tol.order_id=om.id
left join db_digua_business.t_merchant tmer on tmer.id = om.merchant_id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
AND tmer.shop_type !=2  -- 剔除商家

'''

df_order = query(sql1)
df_order.shape

(24072, 40)

# 提取订单风控数据

# 合并数据

In [5]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(24072)

# 处理日期

In [6]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method', 'has_actual',
       'relet_status', 'order_finish_date', 'overdue', 'rembursement_status',
       'purchase_amount', 'delivery_province_name', 'contract_price',
       'rent_received', 'all_rental', 'unearned_rent_revenue'],
      dtype='object')

In [7]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df["下单月份"] = df["create_time"].dt.strftime('%Y-%m')
df.shape


(24072, 43)

In [8]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()


拒绝理由
                                                                                     23108
命中云商分低于515强拒(芝麻综合风险等级0豁免)                                                              543
命中火眸分<550,并且为搜索渠道(自有模型免审豁免)                                                             17
冰鉴火眸分<500命中强拒(芝麻综合风险等级0豁免)                                                              16
命中青云分<500,并且为搜索渠道(自有模型免审豁免)                                                              6
命中火眸分<550,并且为搜索渠道(自有模型免审豁免), 命中青云分<500,并且为搜索渠道(自有模型免审豁免)                                 6
命中融安分低于680强拒(芝麻综合风险等级0豁免)                                                                5
命中火眸分<550,并且为搜索渠道(自有模型免审豁免), 履约历史等级1且为搜索渠道(自有模型免审豁免)                                     4
命中云商分低于515强拒(芝麻综合风险等级0豁免), 命中融安分低于680强拒(芝麻综合风险等级0豁免)                                     2
命中云商分低于515强拒(芝麻综合风险等级0豁免), 冰鉴火眸分<500命中强拒(芝麻综合风险等级0豁免)                                    2
综合风险等级为3且非免押, 蚁盾分>=80                                                                

# 处理备注信息

In [9]:
df = df[df['sku_attributes'].notnull()]


In [10]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].astype(str).str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].astype(str).str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [11]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
1    14207
0     9865
Name: count, dtype: int64

# 渠道归属

In [12]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                   14793
搜索渠道                    4811
未知渠道                    1151
租机宝                      659
单人聊天会话中的小程序消息卡片（分享）      396
                       ...  
618双重狂欢                    1
糖果MAU04                    1
邦道-自定义                     1
邦道-销售线索                    1
邦客惠乘车码                     1
Name: count, Length: 61, dtype: int64

# 订单去重

In [13]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method', 'has_actual',
       'relet_status', 'order_finish_date', 'overdue', 'rembursement_status',
       'purchase_amount', 'delivery_province_name', 'contract_price',
       'rent_received', 'all_rental', 'unearned_rent_revenue', '下单日期', '月份',
       '下单月份', '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型',
       '优惠券使用否', '是否进行预授权', '颜色', '内存', '进件', '来源渠道', '免审', '归属渠道'],
      dtype='object')

In [14]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [15]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [16]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
人审拒绝    24047
机审强拒        9
Name: count, dtype: int64

# 保留商家的数据

In [17]:
df_contain = df.copy()

# 剔除商家数据

In [18]:
#剔除商家数据只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)
df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)  
# 趣智数码  单
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)
# df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)


# 去重订单明细导出

In [19]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2024-10-31'

# 导入服务订单

In [20]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(24053, 64)

In [21]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)



# 导入出库台账数据

In [22]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# wb = pd.ExcelFile(f_path_ck)  
# df_ck = pd.read_excel(wb, sheet_name="2023")
df_ck = pd.read_excel(f_path_ck,sheet_name="2023")   #,skiprows=0 header 
df_ck.shape

(20732, 18)

In [23]:
df_ck['溢价费订单'] = "A"+df_ck[df_ck['备注'].str.contains(pat='溢价费',regex=False)==True]['备注'].str.extract('(\d+.\d+)')
df_yijia = df_ck[df_ck['溢价费订单'].str.len()>=16][['溢价费订单',"已付金额"]]
df_yijia=df_yijia.rename(columns={'溢价费订单':"单号","已付金额":"溢价费"})
df_yijia_ck = pd.merge(df_ck,df_yijia,left_on="订单号", right_on="单号",how="left")
dfck = pd.merge(df_yijia_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape#.to_excel(f"C:/Users/Administrator/Desktop/报表模板/出库台账最终整理/11月出库补充1220.xlsx")

(19391, 95)

In [24]:
dfck.columns

Index(['日期', '订单号', '类目', '型号', '内存_x', '颜色_x', '平台套餐', '租金', '期数', '已付金额',
       '订单归属', '出租方', '备注', '快递单号', '供应商', '采购金额', '碎屏保', '串码', '溢价费订单', '单号',
       '溢价费', 'create_date', 'create_time', 'order_id', 'order_number',
       'all_money', 'status', 'status2', 'channel_type', 'sku_attributes',
       'product_name', 'new_actual_money', 'back_type', 'user_mobile',
       'true_name', 'id_card_num', 'total_describes', 'decision_result',
       'cancel_reason', 'trace_id', 'rejected', 'result', 'channel_name',
       'activity_name', 'merchant_id', 'merchant_name',
       'total_freeze_fund_amount', 'buy_service_product', 'service_status',
       'order_method', 'has_actual', 'relet_status', 'order_finish_date',
       'overdue', 'rembursement_status', 'purchase_amount',
       'delivery_province_name', 'contract_price', 'rent_received',
       'all_rental', 'unearned_rent_revenue', '下单日期', '月份', '下单月份', '拒绝理由',
       '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否', '

In [25]:
dfck[dfck["status2"]=="已退款"]

dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)


In [26]:
df['result'].value_counts()

result
【风控复核】建议商户对租户进行电审，如：如 1.通过核实用户的工作情况和累计贷款金额等，判断偿还能力；2.核实常驻地址；3.增加紧急联系人。为保证订单流程规范，发货前请注意核实租户身份证正反面图片是否正常上传，以便在订单发生坏账时更有利于起诉。提醒租户需要下单人本人带身份证签收。    15965
【风控通过】该用户风控资质较好，建议商户可以优先审核发货。为保证订单流程规范，发货前请注意核实租户身份证正反面图片是否正常上传，以便在订单发生坏账时更有利于起诉。提醒租户需要下单人本人带身份证签收。                                              7125
【风控复核】建议商户对租户进行电审，如：1.通过核实用户的工作情况和累计贷款金额等，判断偿还能力；2.核实常驻地址；3.增加紧急联系人。为保证订单流程规范，发货前请注意核实租户身份证正反面图片是否正常上传，以便在订单发生坏账时更有利于起诉。提醒租户需要下单人本人带身份证签收。        596
                                                                                                                                                   11
【风控拒绝】风控不通过，租户风险较高。建议预收到设备价值或者拒绝。                                                                                                                   9
【优质订单】免人审                                                                                                                                           1
数据校验通过                                                                                       

# 到期订单

In [27]:
df1 = df[(df['下单月份']>='2023-01')&(df['下单月份']<='2024-05')]
df = df[(df['下单月份']>='2023-06')&(df['下单月份']<='2024-04')]
# df['下单月份'].drop_duplicates()

In [28]:

df0 = pd.crosstab(df["下单月份"],df["rembursement_status"],margins=True)
# df0 = df0.merge(df1, on='下单月份', how='left')

df0["逾期率"]=df0["已逾期"]/df0[["已买断","已完成","已逾期","续租中","还款中","检测中"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["买断率"]=df0["已买断"]/df0["All"]
df0["买断率"]=df0["买断率"].apply(lambda x:format(x,".2%"))
df0["归还率"]=(df0["已完成"]+df0["检测中"])/df0["All"]
df0["归还率"]=df0["归还率"].apply(lambda x:format(x,".2%"))
df0["续租率"]=df0["续租中"]/df0["All"]
df0["续租率"]=df0["续租率"].apply(lambda x:format(x,".2%"))

df0.rename(columns={'All': '出库'}, inplace=True)
df0
# with pd.ExcelWriter('F:/需求/总需求/数据统计.xlsx', engine='openpyxl') as writer:
#     df0.to_excel(writer, sheet_name='数据统计')

rembursement_status,已买断,已完成,已逾期,检测中,续租中,还款中,出库,逾期率,买断率,归还率,续租率
下单月份,,,,,,,,,,,
2023-06,107,65,243,8,163,1,587,41.40%,18.23%,12.44%,27.77%
2023-07,159,96,381,5,240,1,882,43.20%,18.03%,11.45%,27.21%
2023-08,153,95,346,5,278,1,878,39.41%,17.43%,11.39%,31.66%
2023-09,159,74,349,10,298,0,890,39.21%,17.87%,9.44%,33.48%
2023-10,206,54,493,16,392,36,1197,41.19%,17.21%,5.85%,32.75%
2023-11,231,10,437,0,184,701,1563,27.96%,14.78%,0.64%,11.77%
2023-12,60,21,278,0,27,609,995,27.94%,6.03%,2.11%,2.71%
2024-01,29,9,179,0,2,481,700,25.57%,4.14%,1.29%,0.29%
2024-02,18,0,82,0,0,277,377,21.75%,4.77%,0.00%,0.00%


In [29]:
# 定义一个函数来计算年龄
def calculate_age(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

# 应用函数到 DataFrame
df['age'] = df.apply(lambda row: calculate_age(row['id_card_num'], row['create_time']), axis=1)
df = df[(df['purchase_amount']>0)&(df['age']>20)]

# 性别分布

In [30]:
# 定义一个函数来判断性别
def get_gender(id_card):
    gender_digit = int(id_card[-2])  # 第17位数字
    return '男' if gender_digit % 2 != 0 else '女'

# 应用函数到 DataFrame
df['性别'] = df['id_card_num'].apply(get_gender)


In [31]:
df[['id_card_num', '性别']]
df_sex = pd.crosstab(df["下单月份"],df["性别"],margins=True)
df_sex['男性占比'] = df_sex['男'] / df_sex['All']
df_sex['男性占比'] = df_sex['男性占比'].apply(lambda x: format(x, '.2%'))
df_sex['女性占比'] = df_sex['女'] / df_sex['All']
df_sex['女性占比'] = df_sex['女性占比'].apply(lambda x: format(x, '.2%'))
df_sex

性别,女,男,All,男性占比,女性占比
下单月份,,,,,
2023-06,71,423,494,85.63%,14.37%
2023-07,104,641,745,86.04%,13.96%
2023-08,132,610,742,82.21%,17.79%
2023-09,135,621,756,82.14%,17.86%
2023-10,219,844,1063,79.40%,20.60%
2023-11,328,1077,1405,76.65%,23.35%
2023-12,239,659,898,73.39%,26.61%
2024-01,216,437,653,66.92%,33.08%
2024-02,74,267,341,78.30%,21.70%


# 年龄分布

In [32]:

# 定义年龄区间
age_bins = [20, 23, 25, 30, 35, 40, 45, float('inf')]
age_labels = ['[20,23)', '[23,25)', '[25,30)', '[30,35)', '[35,40)', '[40,45)', '[45+)']

# 使用 pd.cut 设置年龄区间
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=True, include_lowest=False)

df[['order_number', 'id_card_num', 'age', 'age_group']]
df_age = pd.crosstab(df["下单月份"],df["age_group"],margins=True)
df_age['[20,23)比例'] = df_age['[20,23)'] / df_age['All']
df_age['[20,23)比例'] = df_age['[20,23)比例'].apply(lambda x: format(x, '.2%'))

df_age['[23,25)比例'] = df_age['[23,25)'] / df_age['All']
df_age['[23,25)比例'] = df_age['[23,25)比例'].apply(lambda x: format(x, '.2%'))

df_age['[23,25)比例'] = df_age['[23,25)'] / df_age['All']
df_age['[23,25)比例'] = df_age['[23,25)比例'].apply(lambda x: format(x, '.2%'))

df_age['[23,25)比例'] = df_age['[23,25)'] / df_age['All']
df_age['[23,25)比例'] = df_age['[23,25)比例'].apply(lambda x: format(x, '.2%'))

df_age


age_group,"[20,23)","[23,25)","[25,30)","[30,35)","[35,40)","[40,45)",[45+),All,"[20,23)比例","[23,25)比例"
下单月份,,,,,,,,,,
2023-06,159,80,128,84,29,12,2,494,32.19%,16.19%
2023-07,197,126,205,122,59,25,11,745,26.44%,16.91%
2023-08,165,104,222,138,72,24,17,742,22.24%,14.02%
2023-09,152,141,208,152,65,33,5,756,20.11%,18.65%
2023-10,222,180,310,198,102,39,12,1063,20.88%,16.93%
2023-11,227,209,416,276,165,89,23,1405,16.16%,14.88%
2023-12,122,107,264,173,136,71,25,898,13.59%,11.92%
2024-01,108,88,153,129,124,40,11,653,16.54%,13.48%
2024-02,48,44,98,73,47,25,6,341,14.08%,12.90%


In [33]:
# df[['order_number', 'id_card_num', 'age', 'age_group']].to_excel('F:/sfd.xlsx')

# 地域分布

In [34]:
df[['order_number', 'delivery_province_name']]

df_province1 = pd.crosstab(df["下单月份"],df["delivery_province_name"],margins=True)

df['delivery_province_name'] = np.where(df['delivery_province_name']=='上海', '上海市', 
                                        np.where(df['delivery_province_name']=='北京', '北京市',
                                                np.where(df['delivery_province_name']=='天津', '天津市',
                                                        np.where(df['delivery_province_name']=='重庆', '重庆市', df['delivery_province_name']))))
province_list = ['广东省', '江苏省', '浙江省', '福建省', '山东省', '四川省', '河南省', '河北省', '安徽省', '云南省', '湖北省', '湖南省', '江西省', '陕西省', '上海市', '辽宁省', 
                '北京市', '重庆市', '贵州省', '广西壮族自治区', '山西省', '黑龙江省', '吉林省', '甘肃省', '天津市', '海南省']
df['delivery_province_name'] = np.where(df['delivery_province_name'].isin(province_list), df['delivery_province_name'], '其他')
df_province = pd.crosstab(df["下单月份"],df["delivery_province_name"],margins=True)
df_province = df_province[['广东省','江苏省','浙江省','福建省','山东省','四川省','河南省','河北省','安徽省','云南省','湖北省','湖南省','江西省','陕西省','上海市','辽宁省','北京市',
                                '重庆市', '贵州省','广西壮族自治区','山西省','黑龙江省','吉林省','甘肃省','天津市','海南省','其他', 'All']]


In [35]:
# df_province['广东省占比'] = df_province['广东省'] / df_province['All']
# df_province['广东省占比'] = df_province['广东省占比'].apply(lambda x: format(x, '.2%'))

# df_province['上海市占比'] = df_province['上海市'] / df_province['All']
# df_province['上海市占比'] = df_province['上海市占比'].apply(lambda x: format(x, '.2%'))

# df_province['云南省占比'] = df_province['云南省'] / df_province['All']
# df_province['云南省占比'] = df_province['云南省占比'].apply(lambda x: format(x, '.2%'))

# df_province['北京市占比'] = df_province['北京市'] / df_province['All']
# df_province['北京市占比'] = df_province['北京市占比'].apply(lambda x: format(x, '.2%'))

# df_province['上海市占比'] = df_province['上海市'] / df_province['All']
# df_province['上海市占比'] = df_province['上海市占比'].apply(lambda x: format(x, '.2%'))

# df_province['吉林省占比'] = df_province['吉林省'] / df_province['All']
# df_province['吉林省占比'] = df_province['吉林省占比'].apply(lambda x: format(x, '.2%'))

# df_province['四川省占比'] = df_province['四川省'] / df_province['All']
# df_province['四川省占比'] = df_province['四川省占比'].apply(lambda x: format(x, '.2%'))

# df_province['天津市占比'] = df_province['天津市'] / df_province['All']
# df_province['天津市占比'] = df_province['天津市占比'].apply(lambda x: format(x, '.2%'))

# df_province['安徽省占比'] = df_province['安徽省'] / df_province['All']
# df_province['安徽省占比'] = df_province['安徽省占比'].apply(lambda x: format(x, '.2%'))

# df_province['山东省占比'] = df_province['山东省'] / df_province['All']
# df_province['山东省占比'] = df_province['山东省占比'].apply(lambda x: format(x, '.2%'))

# df_province['山西省占比'] = df_province['山西省'] / df_province['All']
# df_province['山西省占比'] = df_province['山西省占比'].apply(lambda x: format(x, '.2%'))

# df_province['广西壮族自治区占比'] = df_province['广西壮族自治区'] / df_province['All']
# df_province['广西壮族自治区占比'] = df_province['广西壮族自治区占比'].apply(lambda x: format(x, '.2%'))

# df_province['江苏省占比'] = df_province['江苏省'] / df_province['All']
# df_province['江苏省占比'] = df_province['江苏省占比'].apply(lambda x: format(x, '.2%'))

# df_province['江西省占比'] = df_province['江西省'] / df_province['All']
# df_province['江西省占比'] = df_province['江西省占比'].apply(lambda x: format(x, '.2%'))

# df_province['河北省占比'] = df_province['河北省'] / df_province['All']
# df_province['河北省占比'] = df_province['河北省占比'].apply(lambda x: format(x, '.2%'))

# df_province['河南省占比'] = df_province['河南省'] / df_province['All']
# df_province['河南省占比'] = df_province['河南省占比'].apply(lambda x: format(x, '.2%'))

# df_province['浙江省占比'] = df_province['浙江省'] / df_province['All']
# df_province['浙江省占比'] = df_province['浙江省占比'].apply(lambda x: format(x, '.2%'))

# df_province['海南省占比'] = df_province['海南省'] / df_province['All']
# df_province['海南省占比'] = df_province['海南省占比'].apply(lambda x: format(x, '.2%'))

# df_province['湖北省占比'] = df_province['湖北省'] / df_province['All']
# df_province['湖北省占比'] = df_province['湖北省占比'].apply(lambda x: format(x, '.2%'))

# df_province['湖南省占比'] = df_province['湖南省'] / df_province['All']
# df_province['湖南省占比'] = df_province['湖南省占比'].apply(lambda x: format(x, '.2%'))

# df_province['甘肃省占比'] = df_province['甘肃省'] / df_province['All']
# df_province['甘肃省占比'] = df_province['甘肃省占比'].apply(lambda x: format(x, '.2%'))

# df_province['福建省占比'] = df_province['福建省'] / df_province['All']
# df_province['福建省占比'] = df_province['福建省占比'].apply(lambda x: format(x, '.2%'))

# df_province['贵州省占比'] = df_province['贵州省'] / df_province['All']
# df_province['贵州省占比'] = df_province['贵州省占比'].apply(lambda x: format(x, '.2%'))

# df_province['辽宁省占比'] = df_province['辽宁省'] / df_province['All']
# df_province['辽宁省占比'] = df_province['辽宁省占比'].apply(lambda x: format(x, '.2%'))

# df_province['重庆市占比'] = df_province['重庆市'] / df_province['All']
# df_province['重庆市占比'] = df_province['重庆市占比'].apply(lambda x: format(x, '.2%'))

# df_province['陕西省占比'] = df_province['陕西省'] / df_province['All']
# df_province['陕西省占比'] = df_province['陕西省占比'].apply(lambda x: format(x, '.2%'))

# df_province['黑龙江省占比'] = df_province['黑龙江省'] / df_province['All']
# df_province['黑龙江省占比'] = df_province['黑龙江省占比'].apply(lambda x: format(x, '.2%'))

# df_province['其他占比'] = df_province['其他'] / df_province['All']
# df_province['其他占比'] = df_province['其他占比'].apply(lambda x: format(x, '.2%'))

In [36]:
df_province

delivery_province_name,广东省,江苏省,浙江省,福建省,山东省,四川省,河南省,河北省,安徽省,云南省,湖北省,湖南省,江西省,陕西省,上海市,辽宁省,北京市,重庆市,贵州省,广西壮族自治区,山西省,黑龙江省,吉林省,甘肃省,天津市,海南省,其他,All
下单月份,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-06,80,37,45,23,26,30,24,17,25,21,14,14,8,10,19,13,8,16,13,9,9,9,1,2,3,8,10,494
2023-07,117,72,67,35,38,61,38,20,29,23,21,25,21,15,26,15,16,15,16,19,10,8,10,5,6,4,13,745
2023-08,94,78,60,33,41,64,25,26,28,24,21,27,17,22,26,18,10,12,23,21,14,12,7,7,8,4,20,742
2023-09,108,65,76,36,45,43,32,34,33,21,15,31,13,23,32,16,16,11,19,21,10,9,15,7,7,5,13,756
2023-10,132,103,101,41,59,85,47,47,37,39,29,36,24,22,21,26,15,27,36,22,18,17,17,8,13,7,34,1063
2023-11,191,144,130,50,76,111,55,55,41,40,30,43,34,39,37,45,36,24,36,41,39,22,23,12,16,9,26,1405
2023-12,88,90,72,36,52,55,54,43,25,49,16,35,18,23,28,28,16,23,23,26,15,12,9,13,7,12,30,898
2024-01,64,62,51,26,42,40,32,28,29,25,17,24,20,11,17,28,15,11,26,8,14,16,17,6,6,5,13,653
2024-02,42,33,31,17,25,18,17,9,17,9,6,6,8,9,13,9,11,5,11,10,4,11,6,4,4,1,5,341


# 单笔放款金额区间

In [37]:
df[df['purchase_amount']==0]['order_number'].count()

np.int64(0)

In [38]:
# 定义金额区间
price_bins = [0, 3000, 5000, 10000, 20000, 30000]
price_labels = ['(0, 3000)', '[3000,5000)', '[5000,10000)', '[10000,20000)', '[20000,30000)']

# 使用 pd.cut 设置金额区间
df['price_group'] = pd.cut(df['new_actual_money'], bins=price_bins, labels=price_labels, right=True, include_lowest=False)

df[['order_number', 'contract_price', 'price_group']]
df_price = pd.crosstab(df["下单月份"],df["price_group"],margins=True)
df_price['(0, 3000)比例'] = df_price['(0, 3000)'] / df_price['All']
df_price['(0, 3000)比例'] = df_price['(0, 3000)比例'].apply(lambda x: format(x, '.2%'))

df_price['[3000,5000)比例'] = df_price['[3000,5000)'] / df_price['All']
df_price['[3000,5000)比例'] = df_price['[3000,5000)比例'].apply(lambda x: format(x, '.2%'))

df_price['[5000,10000)比例'] = df_price['[5000,10000)'] / df_price['All']
df_price['[5000,10000)比例'] = df_price['[5000,10000)比例'].apply(lambda x: format(x, '.2%'))

df_price['[10000,20000)比例'] = df_price['[10000,20000)'] / df_price['All']
df_price['[10000,20000)比例'] = df_price['[10000,20000)比例'].apply(lambda x: format(x, '.2%'))

df_price

price_group,"(0, 3000)","[3000,5000)","[5000,10000)","[10000,20000)",All,"(0, 3000)比例","[3000,5000)比例","[5000,10000)比例","[10000,20000)比例"
下单月份,,,,,,,,,
2023-06,10,18,100,366,494,2.02%,3.64%,20.24%,74.09%
2023-07,12,20,109,604,745,1.61%,2.68%,14.63%,81.07%
2023-08,10,16,118,598,742,1.35%,2.16%,15.90%,80.59%
2023-09,6,20,132,598,756,0.79%,2.65%,17.46%,79.10%
2023-10,13,22,268,760,1063,1.22%,2.07%,25.21%,71.50%
2023-11,19,13,379,994,1405,1.35%,0.93%,26.98%,70.75%
2023-12,10,14,189,685,898,1.11%,1.56%,21.05%,76.28%
2024-01,16,8,146,483,653,2.45%,1.23%,22.36%,73.97%
2024-02,8,12,135,186,341,2.35%,3.52%,39.59%,54.55%
